In [ ]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    BertForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_function(example):
    question_string = f"What is the sentiment of aspect: {example['aspect']}?"

    encoding = tokenizer(
        text=example["text"],
        text_pair=question_string,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.681411,0.752778
2,0.852300,0.565746,0.777778
3,0.532600,0.599870,0.790278
4,0.327500,0.687765,0.766667
5,0.327500,0.883457,0.781944
6,0.192500,1.068816,0.795833
7,0.120900,1.177420,0.790278
8,0.051400,1.244848,0.788889
9,0.040300,1.344539,0.786111
10,0.040300,1.369334,0.784722


TrainOutput(global_step=1580, training_loss=0.27029807703404485, metrics={'train_runtime': 1395.7993, 'train_samples_per_second': 18.061, 'train_steps_per_second': 1.132, 'total_flos': 3316544630461440.0, 'train_loss': 0.27029807703404485, 'epoch': 10.0})

In [ ]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.0688155889511108, 'eval_accuracy': 0.7958333333333333, 'eval_runtime': 9.3582, 'eval_samples_per_second': 76.938, 'eval_steps_per_second': 4.809, 'epoch': 10.0}


In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.0330814123153687, 'eval_accuracy': 0.8033240997229917, 'eval_runtime': 4.8149, 'eval_samples_per_second': 74.975, 'eval_steps_per_second': 4.777, 'epoch': 10.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA")

('/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA/vocab.txt',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA/added_tokens.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA/tokenizer.json')

In [ ]:
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/bert-base-multilingual-casedQA")

## Vyhodnocovací metriky

In [ ]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [ ]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.0330814123153687, 'eval_model_preparation_time': 0.0079, 'eval_accuracy': 0.8033240997229917, 'eval_runtime': 6.0819, 'eval_samples_per_second': 59.357, 'eval_steps_per_second': 7.563}


In [ ]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [ ]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.0330814123153687, 'eval_model_preparation_time': 0.0088, 'eval_f1': 0.8047469216150598, 'eval_runtime': 6.094, 'eval_samples_per_second': 59.238, 'eval_steps_per_second': 7.548}


In [ ]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [ ]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.0330814123153687, 'eval_model_preparation_time': 0.005, 'eval_f1': 0.7536059754279002, 'eval_runtime': 6.2769, 'eval_samples_per_second': 57.512, 'eval_steps_per_second': 7.328}


## Praktické testy

In [ ]:
def predict_sentiment(model, tokenizer, aspect, text):
    question_string = f"What is the sentiment of aspect: {aspect}?"

    encoding = tokenizer(
        text=text,
        text_pair=question_string,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [ ]:
example_aspect = "power"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "display"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "battery"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-2.0458, -2.8137,  5.0639]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 4.1768, -2.8387, -1.3503]])
Predicted sentiment for L2: negative, expected: negative
tensor([[-2.1747, -2.7448,  5.1267]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [ ]:
example_aspect = "staff"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "table"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "food"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "beer"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 4.5744, -2.3303, -2.1564]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 4.5842, -1.9690, -2.5232]])
Predicted sentiment for R2: negative, expected: negative
tensor([[-1.8460, -2.5989,  4.5832]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-2.2217, -2.7217,  5.1965]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [ ]:
example_aspect = "Osaka"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[ 1.8956,  0.5020, -2.5259]])
Predicted sentiment for M1: negative, expected: neutral
tensor([[ 1.7236, -2.0636,  0.5278]])
Predicted sentiment for M2: negative, expected: positive
tensor([[ 4.5277, -1.9126, -2.5553]])
Predicted sentiment for M3: negative, expected: negative


### Sada O

In [ ]:
example_aspect = "day"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "weather"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[ 0.0089, -2.8283,  3.0006]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 4.5475, -2.1768, -2.2993]])
Predicted sentiment for O2: negative, expected: negative
